In [1]:
from datasets import load_dataset
import datasets
import torchaudio

/home/sanchitgandhi/hf/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
dataset_cache_dir = "/home/sanchitgandhi/cache/huggingface/datasets"

In [3]:
swbd = load_dataset("/home/sanchitgandhi/datasets/datasets/switchboard", "switchboard", split="train[:1000]", cache_dir=dataset_cache_dir, use_auth_token=True, ignore_verifications=True)

Reusing dataset switchboard (/home/sanchitgandhi/cache/huggingface/datasets/switchboard/switchboard/1.1.0/dae178d6d13fe90241277e548542a56b7db74c8ce4d14acff28905b2cbb89f4a)


In [4]:
swbd

Dataset({
    features: ['audio', 'text', 'file', 'id'],
    num_rows: 1000
})

In [5]:
swbd_resampled = swbd.cast_column(
        "audio", datasets.features.Audio(sampling_rate=16_000)
    )

def prepare_dataset(batch):
    sample = batch["audio"]
    batch["resampled_inputs"] = sample["array"]
    batch["resampled_sampling_rate"] = sample["sampling_rate"]
    return batch

In [6]:
%%time

swbd_resampled = swbd_resampled.map(
        prepare_dataset,
        remove_columns=swbd_resampled.column_names,
        num_proc=1,
        desc="preprocess train dataset",
)

preprocess train dataset:   0%|                                                                | 0/1000 [00:00<?, ?ex/s]/home/sanchitgandhi/datasets/src/datasets/features/audio.py:278: FutureWarning: Pass orig_sr=8000, target_sr=16000 as keyword args. From version 0.10 passing these as positional arguments will result in an error
  array = librosa.resample(array, sampling_rate, self.sampling_rate, res_type="kaiser_best")
preprocess train dataset:   0%|                                                        | 1/1000 [00:01<22:47,  1.37s/ex]/home/sanchitgandhi/datasets/src/datasets/features/audio.py:278: FutureWarning: Pass orig_sr=8000, target_sr=16000 as keyword args. From version 0.10 passing these as positional arguments will result in an error
  array = librosa.resample(array, sampling_rate, self.sampling_rate, res_type="kaiser_best")
preprocess train dataset: 100%|█████████████████████████████████████████████████████| 1000/1000 [01:17<00:00, 12.96ex/s]


CPU times: user 1min 17s, sys: 6.55 s, total: 1min 24s
Wall time: 1min 18s


In [7]:
resampler = torchaudio.transforms.Resample(8_000, 16_000)

def prepare_dataset(batch):
    sample = batch["audio"]
    batch["resampled_inputs"] = sample["array"]
    batch["resampled_sampling_rate"] = sample["sampling_rate"]
    return batch

In [11]:
%%time

swbd_resampled_torchaudio = swbd.map(
        prepare_dataset,
        remove_columns=swbd.column_names,
        num_proc=1,
        desc="preprocess train dataset",
)

preprocess train dataset: 100%|████████████████████████████████████████████████████| 1000/1000 [00:01<00:00, 558.53ex/s]


CPU times: user 835 ms, sys: 1.1 s, total: 1.93 s
Wall time: 2.39 s


Librosa: 78s

torchaudio: 2.39s (32x faster)